In [1]:
import pandas as pd
import os

In [2]:
# Combine separate airline sheets into one table. All sheets have identical columns.
files = ['airline_destination_data/'+x for x in os.listdir('airline_destination_data/') if ".csv" in x]
airline_table = pd.DataFrame(columns=['Airline', 'Notes', 'icao'])
for file in files:
    airline_table = airline_table.append(pd.read_csv(file))

In [3]:
# ICAO column is named 'ident' in the geocoded table.
airline_table = airline_table.rename(columns={'icao':'ident'})

# Join airports and airlines on ICAO code. This will generate duplicate rows that we will use to fill the Y/N table.
airports_with_airlines = pd.merge(pd.read_csv('geodata/airports.csv'), airline_table, on=['ident'])

In [4]:
# Create a new column for each airline, with N value by default.
for airline in airports_with_airlines.Airline.unique():
    airports_with_airlines[airline] = "N"

In [5]:
# Mark Y in each airline's column if they reach each airport.
for airport in airports_with_airlines.name.unique():
    airport_frame = airports_with_airlines[airports_with_airlines.name == airport]
    for airline in airport_frame.Airline.unique():
        for idx in airport_frame.index:
            airports_with_airlines[airline][idx] = "Y"

# Now that there is a column for each airline, we can drop duplicates by airport.
airports_with_airlines = airports_with_airlines.drop_duplicates(subset=['name']).reset_index()

<ipython-input-5-0bea6536d6d1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports_with_airlines[airline][idx] = "Y"


In [6]:
# Count the number of airlines that reach each airport. Rename ident to ICAO for an upcoming join.
airports_with_airlines['airline_count'] = 0
for row in airports_with_airlines.index:
    airports_with_airlines.loc[row, 'airline_count'] = \
    airports_with_airlines.iloc[row].tolist().count("Y")
airports_with_airlines = airports_with_airlines.rename(columns={'ident':'ICAO'})

In [7]:
# Include hubs from data gathered and organized into hubs.csv
hubs_included = pd.merge(pd.read_csv('airline_destination_data/hubs/hubs.csv'), airports_with_airlines, on=['ICAO'], how='outer')

In [8]:
# Reorganize columns for readability
hubs_included = hubs_included[['id', 'ICAO',
       'type', 'name', 'Weight', 'latitude_deg', 'longitude_deg', 'elevation_ft',
       'continent', 'iso_country', 'iso_region', 'municipality','Alaska Airlines', 'American Airlines',
       'Allegiant Air', 'Frontier Airlines', 'JetBlue Airways',
       'Delta Air Lines', 'United Airlines', 'SkyWest Airlines',
       'Southwest Airlines', 'Hawaiian Airlines', 'airline_count']]
hubs_included

In [10]:
hubs_included.to_csv('geodata/airline_master_yntable.csv', index=False)